# Graph Convolutional Network
Proof of concept using PyTorch Geometric (PyG)
- Task: graph classification; given game state, who will win?
- Data: x: (mobility graph @ my turn) y: (did i win?)
- Network: ???

## Data Prep
- Each move of games minus first 10 and last 10
- Split into black and white (doesn't handle prediction when other player's turn)

In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
import torch_geometric as tg
import chessdb
from data import get_graphs
import tqdm
from torch_geometric.utils.convert import from_networkx
import networkx as nx
import random
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import torch

In [24]:
### use list -> dataloader
df = chessdb.parse(nrows=10)
games = df.apply(lambda row: chessdb.get_game(row), axis=1).to_list()
random.shuffle(games)
train_games = games[len(games) // 5:]
test_games = games[:len(games) // 5]

train_graphs = []
for game in tqdm.tqdm(train_games):
    for graph, win in get_graphs(game):
        graph = from_networkx(graph)
        graph.y = win
        train_graphs.append(graph)

test_graphs = []
for game in tqdm.tqdm(test_games):
    for graph, win in get_graphs(game):
        graph = from_networkx(graph)
        graph.y = win
        test_graphs.append(graph)

100%|██████████| 2/2 [00:00<00:00, 23.84it/s]


In [25]:
print(f'Number of graphs: {len(train_graphs + test_graphs)}')

data = train_graphs[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

print('=============================================================')
print(f'Number of training graphs: {len(train_graphs)}')
print(f'Number of test graphs: {len(test_graphs)}')

Number of graphs: 285

Data(edge_index=[2, 36], piece=[64], num_nodes=64, y=True)
Number of nodes: 64
Number of edges: 36
Average node degree: 0.56
Has isolated nodes: True
Has self-loops: False
Is undirected: False
Number of training graphs: 223
Number of test graphs: 62


In [26]:
train_loader = tg.loader.DataLoader(train_graphs, batch_size=64, shuffle=True)
test_loader = tg.loader.DataLoader(test_graphs, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2385], piece=[64], num_nodes=4096, y=[64], batch=[4096], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2456], piece=[64], num_nodes=4096, y=[64], batch=[4096], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 2380], piece=[64], num_nodes=4096, y=[64], batch=[4096], ptr=[65])

Step 4:
Number of graphs in the current batch: 31
DataBatch(edge_index=[2, 1088], piece=[31], num_nodes=1984, y=[31], batch=[1984], ptr=[32])



In [30]:
class GCN(torch.nn.Module):
    def __init__(self, n_node_features=1, hidden_channels=64, n_cls=2):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(n_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, n_cls)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(1, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
# ### torch_geometric.data.InMemoryDataset or Dataset
# FPATH = '/home/asy51/repos/graphmaster/dataset/all_with_filtered_anotations_since1998.txt'
# class ChessDataset(tg.data.Dataset):
#     def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
#         super().__init__(root, transform, pre_transform, pre_filter)
#         self.data, self.slices = torch.load(self.processed_paths[0])

#     @property
#     def raw_file_names(self):
#         return [FPATH]

#     @property
#     def processed_file_names(self):
#         return ['data.pt']

#     def download(self):
#         # Download to `self.raw_dir`.
#         download_url(url, self.raw_dir)
#         ...

#     def process(self):
#         # Read data into huge `Data` list.
#         data_list = [...]

#         if self.pre_filter is not None:
#             data_list = [data for data in data_list if self.pre_filter(data)]

#         if self.pre_transform is not None:
#             data_list = [self.pre_transform(data) for data in data_list]

#         data, slices = self.collate(data_list)
#         torch.save((data, slices), self.processed_paths[0])